Developing a populator for our RegMagic bias calculation. 

We need to populate both redmapper and redmagic HODs, then project them down while keeping track of galaxies. 

This just handles the joint population. 

Currently using pearce, but we may want to make this pure halotools. 

In [1]:
from pearce.mocks.kittens import cat_dict
from halotools.empirical_models import Zheng07Cens, Zheng07Sats
import numpy as np

In [2]:
a = 0.81120
cosmo_params = {'simname':'chinchilla', 'Lbox':400.0, 'scale_factors':[a]}

In [3]:
cat = cat_dict[cosmo_params['simname']](**cosmo_params)#construct the specified catalog!

In [4]:
cat.load(a, tol = 0.01, HOD='hsabRedMagic', particles = False)#, hod_kwargs = {'sec_haloprop_key':'halo_log_nfw_conc'})#, hod_kwargs={'split': 0.5})

In [5]:
MAP = [12.3475794,   0.26163006,  0.30944855, 11.41793147,  0.44999606,  0.01446294,
 13.94853439,  1.20729256]

param_names = ['logMmin','mean_occupation_centrals_assembias_param1', 'f_c', 'logM0', 'sigma_logM',
                                     'mean_occupation_satellites_assembias_param1',     'logM1',   'alpha']

params = dict(zip(param_names, MAP))

In [6]:
cat.populate(params)

/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:205: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]
/u/ki/swmclau2/.local/lib/python2.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/monte_carlo_helpers.py:522: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  self.rad_prof_func_table_indices[digitized_param_list]


In [7]:
redmagic_table = cat.model.mock.galaxy_table

In [8]:
class RedMapperCens(Zheng07Cens):
    '''Trivial RedMapper HOD. '''

    def __init__(self, **kwargs):
        super(RedMapperCens, self).__init__(**kwargs)

        # Default values from our analysis
        # has no params
        self.param_dict = {}#.update(defaults)  # overwrite halotools zheng07 defaults with our own

    def mean_occupation(self, **kwargs):
        '''See Zheng07 for details.'''
         
        # Retrieve the array storing the mass-like variable
        if 'table' in list(kwargs.keys()):
            mass = kwargs['table'][self.prim_haloprop_key]
        elif 'prim_haloprop' in list(kwargs.keys()):
            mass = np.atleast_1d(kwargs['prim_haloprop'])
        else:
            msg = ("\nYou must pass either a ``table`` or ``prim_haloprop`` argument \n"
                   "to the ``mean_occupation`` function of the ``Zheng07Sats`` class.\n")
            raise HalotoolsError(msg)
        
        min_mass = 5e13
        over_idxs = mass > min_mass
        mean_ncen = np.zeros_like(mass)
        mean_ncen[over_idxs] = 1.0
        return mean_ncen # all RedMapper clusters have centrals, for the masses we're interested in 

In [9]:
class RedMapperSats(Zheng07Sats):
    '''Tweak of Zheng model to add a new parameter, f_c, denoting a modified central fraction.'''

    def __init__(self, cenocc_model, **kwargs):
        # We define modulation to be true. An existing central model is required.
        super(RedMapperSats, self).__init__(modulate_with_cenocc=True, cenocc_model=cenocc_model, **kwargs)
        defaults = {'logM0': 0.0, 'logM1': 13.7, 'alpha': 1.02}
        self.param_dict.update(defaults)
        # It does not like that we have parameters defined multiple places.
        # Required for the central occupations.
        self._suppress_repeated_param_warning = True

    def mean_occupation(self, **kwargs):
        "See Zheng07 for details"
        self.param_dict['logM0'] = 0.0 #defined to be 0
        # could be an issue there

        return super(RedMapperSats, self).mean_occupation(**kwargs)

In [10]:
cat.load_model(a, HOD = (RedMapperCens, RedMapperSats))

In [11]:
MAP = [13.94853439,  1.20729256]

param_names = ['logM1',   'alpha']

params = dict(zip(param_names, MAP))

In [12]:
cat.populate(params)

In [13]:
cat.model.param_dict

{'alpha': 1.20729256, 'logM0': 0.0, 'logM1': 13.94853439}

In [14]:
redmapper_table = cat.model.mock.galaxy_table

In [15]:
redmapper_table

halo_upid,halo_hostid,conc_NFWmodel,halo_y,halo_x,halo_z,halo_vx,halo_vy,halo_vz,halo_rvir,halo_mvir,halo_id,halo_num_centrals,halo_num_satellites,gal_type,vx,host_centric_distance,vy,y,x,vz,z
int64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,int32,int32,object,float32,float64,float32,float32,float32,float32,float32
-1,1447933572,4.3275557,77.76408,63.62292,241.76508,-419.37,-115.11,-136.1,2.241059,1031000000000000.0,1447933572,1,24,centrals,-419.37,0.0,-115.11,77.76408,63.62292,-136.1,241.76508
-1,1447857613,3.7503612,71.19,14.17354,204.80847,391.32,-80.75,611.5,1.513612,317500000000000.0,1447857613,1,5,centrals,391.32,0.0,-80.75,71.19,14.17354,611.5,204.80847
-1,1447404139,3.504818,19.97684,57.73826,251.97795,-166.29,-55.23,92.81,2.0720272,814600000000000.0,1447404139,1,16,centrals,-166.29,0.0,-55.23,19.97684,57.73826,92.81,251.97795
-1,1447938520,4.8521743,48.5323,27.05964,217.8293,-133.2,33.32,675.21,1.60491,378500000000000.0,1447938520,1,4,centrals,-133.2,0.0,33.32,48.5323,27.05964,675.21,217.8293
-1,1447332362,3.5551918,29.94991,43.79602,246.13686,176.81,-118.02,-108.25,1.536938,332400000000000.0,1447332362,1,3,centrals,176.81,0.0,-118.02,29.94991,43.79602,-108.25,246.13686
-1,1448003507,2.4538229,59.09885,5.70921,261.27466,191.44,-194.77,-186.53,1.1537089,140600000000000.0,1448003507,1,3,centrals,191.44,0.0,-194.77,59.09885,5.70921,-186.53,261.27466
-1,1447188320,6.3738856,13.59625,6.3488,126.37466,252.22,590.23,357.66,1.973132,703400000000000.0,1447188320,1,12,centrals,252.22,0.0,590.23,13.59625,6.3488,357.66,126.37466
-1,1447706989,5.994402,64.36566,17.48224,92.56943,-425.41,-211.39,370.02,1.6791099,433500000000000.0,1447706989,1,13,centrals,-425.41,0.0,-211.39,64.36566,17.48224,370.02,92.56943
-1,1447109270,8.246616,34.2258,2.00261,124.24074,-31.24,-47.53,-387.93,1.216112,164700000000000.0,1447109270,1,3,centrals,-31.24,0.0,-47.53,34.2258,2.00261,-387.93,124.24074
